<a href="https://colab.research.google.com/github/yugpsyfer/Playing_with_PyTorch/blob/main/Deep_Learning_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ONLY REQUIRED ONCE

In [ ]:
# !pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

# Import necessary libraries

In [ ]:
import torch
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn
import  torch.nn.functional as F
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***root*** stands for the root folder where you would want to save the MNIST dataset

In [ ]:
root = '' #The folder where MNIST data will be downloaded and saved

# dataset = MNIST(root=root, download=True)
test_dataset = MNIST(root=root, train=False)
dataset = MNIST(root=root,train=True,transform=transforms.ToTensor())

In [ ]:
img_tensor, label = dataset[0]
print(img_tensor.shape,label)

torch.Size([1, 28, 28]) 5


Split the dataset\
Here I have taken a split 50K as train and 10K as test\
The batch size here is 128, since SGD is being used as the optimizer\
*input_size* determines the input layer size\
*num_classes* is for categories, Mnist has multiple categories

In [ ]:
train_ds,test_ds = random_split(dataset,[50000,10000])
batch_size = 128
train_loader = DataLoader(train_ds,batch_size,shuffle=True)
test_loader = DataLoader(test_ds,batch_size)
input_size = 28*28
num_classes= 10

In [ ]:
class MNISTmodel(nn.Module):
  def __init__(self,in_feature,out_feature,learning_rate,optimization_funtion,train_batches,test_batches,hidden_nodes,epochs=100):
    super().__init__()
    self.linear_layer_1 = nn.Linear(in_feature,hidden_nodes)
    self.linear_layer_2 = nn.Linear(hidden_nodes,out_feature)
    self.learning_rate = learning_rate
    self.optimization_funtion = optimization_funtion
    self.epochs = epochs
    self.train_batches = train_batches
    self.test_batches = test_batches

  """Forward method finds the probabilty of each image w.r.t each category and returns it"""
  def forward(self,x):    
    x = x.reshape(-1,784)
    out = self.linear_layer_1(x)
    out = F.relu(out)
    p = self.linear_layer_2(out)
    return p
  
  """Cross Entropy is generally used at a place where multiple classes are present, every input is in a form of minibatch and output is also a minibatch"""
  def __training_step(self,batch):
    loss=0
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    return loss

  def __validation_step(self,batch):
    images, labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = self.__accuracy(out,labels)
    return {0:loss,1:acc}
  
  def __accuracy(self,out,label):
    _,pred = torch.max(out,dim=1)
    return torch.sum(label==pred).item() / len(pred)

  def fit(self):
    optimizer = self.optimization_funtion(self.parameters(),self.learning_rate)
    for epoch in range(self.epochs):
      for batch in self.train_batches:
        loss = self.__training_step(batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

      if epoch%10==0:   #test how good the model is doing after every 10 epochs
        loss=0
        acc=0
        for batch in self.test_batches:
          o_p = self.__validation_step(batch)
          loss += o_p[0]
          acc += o_p[1]
        loss=loss/len(self.test_batches)
        acc =acc/len(self.test_batches)
        print('Loss is {0:1.4f} and Accuracy {1:1.4f} of the model at epoch {2}'.format(loss,acc,epoch))
        print("===================================================================================")


In [ ]:

# MM.fit()

In [ ]:
for i in MM.parameters():
  print(i.shape)

torch.Size([32, 784])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device('cuda')

##Load the data in the GPU

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
#Generator can yield the batches on demand.
train_loader = DeviceDataLoader(train_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

to_device(MM, device)

MNISTmodel(
  (linear_layer_1): Linear(in_features=784, out_features=32, bias=True)
  (linear_layer_2): Linear(in_features=32, out_features=10, bias=True)
)

In [ ]:
MM = MNISTmodel(in_feature=input_size,
                out_feature=num_classes,
                learning_rate=1e-3,
                optimization_funtion=torch.optim.SGD,
                train_batches = train_loader,
                test_batches = test_loader,
                hidden_nodes=32)
to_device(MM, device)
MM.fit()

Loss is 2.2506 and Accuracy 0.1895 of the model at epoch 0
Loss is 1.2266 and Accuracy 0.7689 of the model at epoch 10
Loss is 0.7093 and Accuracy 0.8365 of the model at epoch 20
Loss is 0.5448 and Accuracy 0.8646 of the model at epoch 30
Loss is 0.4679 and Accuracy 0.8804 of the model at epoch 40
Loss is 0.4237 and Accuracy 0.8886 of the model at epoch 50
Loss is 0.3953 and Accuracy 0.8940 of the model at epoch 60
Loss is 0.3754 and Accuracy 0.8970 of the model at epoch 70
Loss is 0.3604 and Accuracy 0.8992 of the model at epoch 80
Loss is 0.3486 and Accuracy 0.9018 of the model at epoch 90
